In [1]:
import pandas as pd
from pathlib import Path

def concat_analysis_frames(root_dir: str, output_path: str):
    root_path = Path(root_dir)
    # Find every file named "analysis_frame.xlsx" under root_dir
    all_analysis_files = list(root_path.rglob("analysis_frame.xlsx"))

    dfs = []
    for analysis_file in all_analysis_files:
        method_folder = analysis_file.parent.parent
        experiment_folder = analysis_file.parent.parent.parent
        
        # Only process if the experiment folder starts with "cluster_"
        if not experiment_folder.name.startswith("cluster_"):
            continue
        
        # Extract datetime string from experiment folder name
        # e.g. "cluster_experiment_results_17022025125138" → last underscore part is "17022025125138"
        parts = experiment_folder.name.split('_')
        if len(parts) > 1:
            datetime_str = parts[-1]
        else:
            datetime_str = "N/A"  # fallback if no underscore

        # Cluster method is simply the parent folder name
        # e.g. "gmm_credit_risk", "kmeans_credit_risk"
        cluster_method = method_folder.name

        # Read the Excel
        df = pd.read_excel(analysis_file)

        # Add identifying columns
        df["cluster_method"] = cluster_method
        df["datetime_str"] = datetime_str

        dfs.append(df)

    # Concatenate and save
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df.to_excel(output_path, index=False)
        print(f"Combined dataframe saved to: {output_path}")
    else:
        print("No matching analysis_frame.xlsx files found under the given root directory.")

concat_analysis_frames(
    root_dir="../experiments",
    output_path="combined_analysis_frames.xlsx"
)


Combined dataframe saved to: combined_analysis_frames.xlsx


In [2]:
import pandas as pd
from pathlib import Path


def filter_df(df, filter_type):
    filtered_df = df[df["Data"].str.contains(filter_type, case=False, na=False)]
    return filtered_df


def concat_filter_analysis_frames(root_dir: str, output_path: str, filter_type: str):
    root_path = Path(root_dir)
    # Find every file named "analysis_frame.xlsx" under root_dir
    all_analysis_files = list(root_path.rglob("analysis_frame.xlsx"))

    dfs = []
    for analysis_file in all_analysis_files:
        method_folder = analysis_file.parent.parent
        experiment_folder = analysis_file.parent.parent.parent
        
        # Only process if the experiment folder starts with "cluster_"
        if not experiment_folder.name.startswith("cluster_"):
            continue
        
        # Extract datetime string from experiment folder name
        # e.g. "cluster_experiment_results_17022025125138" → last underscore part is "17022025125138"
        parts = experiment_folder.name.split('_')
        if len(parts) > 1:
            datetime_str = parts[-1]
        else:
            datetime_str = "N/A"  # fallback if no underscore

        # Cluster method is simply the parent folder name
        # e.g. "gmm_credit_risk", "kmeans_credit_risk"
        cluster_method = method_folder.name

        # Read the Excel
        df = pd.read_excel(analysis_file)

        # Add identifying columns
        df["cluster_method"] = cluster_method
        df["datetime_str"] = datetime_str

        dfs.append(df)

    # Concatenate and save
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)

        filtered_df = filter_df(combined_df, filter_type)

        filtered_df.to_excel(output_path, index=False)
        print(f"Combined dataframe saved to: {output_path}")
    else:
        print("No matching analysis_frame.xlsx files found under the given root directory.")

concat_filter_analysis_frames(
    root_dir=".",
    output_path="train_combined_analysis_frames.xlsx",
    filter_type='train'
)


Combined dataframe saved to: train_combined_analysis_frames.xlsx


In [3]:
# %% [code]
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load your Excel file (change the filename/path as needed)
df = pd.read_excel("combined_analysis_frames.xlsx")

# (Optional) Quick look at the data
df.head()


,AUC Score,Accuracy,Precision,Recall (TPR),F1 Score,Log Loss,Brier Score,Cluster Size,Pure Cluster,Cluster ID,Data,cluster_method,datetime_str
0,0.999996,0.998765,0.997175,0.997175,0.997175,0.005907,0.001093,1620,0.0,0.0,train_xper_scores,kmedoids_credit_risk,17022025212918
1,1.000000,1.000000,1.000000,1.000000,1.000000,0.003780,0.000370,1580,0.0,1.0,train_xper_scores,kmedoids_credit_risk,17022025212918
2,0.862410,0.764068,0.891304,0.195238,0.320312,0.490886,0.158693,1475,0.0,0.0,train_feature_scores,kmedoids_credit_risk,17022025212918
3,0.899706,0.871884,0.000000,0.000000,0.000000,0.315117,0.091416,1725,0.0,1.0,train_feature_scores,kmedoids_credit_risk,17022025212918
4,0.999848,0.988758,1.000000,0.965679,0.982540,0.061740,0.012057,1957,0.0,0.0,train_epsilon_scores,kmedoids_credit_risk,17022025212918


In [4]:
# %% [code]
# Create an Experiment_Type column based on the Data identifier.
df['Experiment_Type'] = df['Data'].apply(lambda x: 'Baseline' if 'baseline' in x.lower() else 'Clustering')

# Extract weighted average rows (assuming the word "weighted" appears in the Data column)
weighted_df = df[df["Data"].str.contains("weighted", case=False, na=False)]


In [5]:
df1 = df[df["Data"].str.contains("weighted", case=False, na=False)]

In [6]:
fig1 = px.box(df1, x="cluster_method", y="AUC Score",
              title="Distribution of AUC Score by Cluster Method")
fig1.show()


In [7]:
# %% [code]
fig2 = px.box(df1, x="cluster_method", y="Recall (TPR)",
              title="Distribution of Recall (TPR) by Cluster Method")
fig2.show()


In [8]:
# %% [code]
fig3 = px.box(df1, x="cluster_method", y="Log Loss",
              title="Distribution of Log Loss by Cluster Method")
fig3.show()


In [9]:
# %% [code]
fig4 = px.box(df1, x="cluster_method", y="Brier Score",
              title="Distribution of Brier Score by Cluster Method")
fig4.show()


In [10]:
df2 = df[(df["Data"].str.contains("test", case=False, na=False)) & ~(df["Data"].str.contains("weighted", case=False, na=False))]

In [11]:
# %% [code]
fig5 = px.scatter(df2, x="Cluster Size", y="AUC Score", color="cluster_method",
                  title="Cluster Size vs. AUC Score by Cluster Method")
fig5.show()


In [12]:
# %% [code]
fig6 = px.scatter(df2, x="Cluster Size", y="Recall (TPR)", color="cluster_method",
                  title="Cluster Size vs. Recall (TPR) by Cluster Method")
fig6.show()


In [13]:
# %% [code]
fig7 = px.scatter(df2, x="Cluster Size", y="Log Loss", color="cluster_method",
                  title="Cluster Size vs. Log Loss by Cluster Method")
fig7.show()


In [14]:
# %% [code]
fig8 = px.scatter(df2, x="Cluster Size", y="Brier Score", color="cluster_method",
                  title="Cluster Size vs. Brier Score by Cluster Method")
fig8.show()


In [15]:
# %% [code]
pure_counts = df2.groupby("cluster_method")["Pure Cluster"].mean().reset_index()
fig11 = px.bar(pure_counts, x="cluster_method", y="Pure Cluster",
               title="Proportion of Pure Clusters by Cluster Method",
               labels={"Pure Cluster": "Proportion of Pure Clusters"})
fig11.show()


In [16]:
# %% [code]
pure_counts = df[df["Data"].str.contains("train", case=False, na=False)].groupby("cluster_method")["Pure Cluster"].mean().reset_index()
fig11 = px.bar(pure_counts, x="cluster_method", y="Pure Cluster",
               title="Proportion of Pure Clusters by Cluster Method",
               labels={"Pure Cluster": "Proportion of Pure Clusters"})
fig11.show()


In [17]:
# %% [code]
fig12 = px.box(df.drop_duplicates(subset=['Data', 'datetime_str']), x="Experiment_Type", y="AUC Score", color="Experiment_Type",
               title="AUC Score Distribution: Baseline vs. Clustering")
fig12.show()


In [18]:
# %% [code]
fig13 = px.scatter_matrix(df2, dimensions=["AUC Score", "Recall (TPR)", "Log Loss", "Brier Score"],
                            color="cluster_method",
                            title="Scatter Matrix of Performance Metrics")
fig13.update_traces(diagonal_visible=False)
fig13.show()


In [19]:
# %% [code]
fig14 = px.box(df[(df['datetime_str'].isin([17022025212918, 17022025125138])) & (df1["Data"] == 'weighted_average_test_xper_scores')], x="Kernel", y="AUC Score", color="Kernel",
               title="AUC Score Distribution: Kernel vs. Non-Kernel Experiments")
fig14.show()


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['AUC Score', 'Accuracy', 'Precision', 'Recall (TPR)', 'F1 Score', 'Log Loss', 'Brier Score', 'Cluster Size', 'Pure Cluster', 'Cluster ID', 'Data', 'cluster_method', 'datetime_str', 'Experiment_Type'] but received: Kernel

In [68]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

def extract_experiment_type(x):
    x_lower = str(x).lower()
    if "xper" in x_lower:
        return "xper"
    elif "feature" in x_lower:
        return "feature"
    elif "epsilon" in x_lower or "error" in x_lower:
        return "error"
    elif "baseline" in x_lower:
        return "baseline"
    else:
        return "other"

df["Exp_Type"] = df["Data"].apply(extract_experiment_type)

# Create filtered DataFrames for later analyses:
#  - Non-weighted, non-pure rows for direct experiment comparisons:
df_clean = df[
    (~df["Data"].str.contains("weighted", case=False, na=False)) &
    (df["Pure Cluster"] == False) &
    (df["Exp_Type"].isin(["xper", "feature", "error"]))
].copy()

#  - Baseline rows
df_baseline = df[df["Data"].str.contains("baseline", case=False, na=False)].copy()
df_baseline["Exp_Type"] = df_baseline["Data"].apply(extract_experiment_type)

#  - Weighted rows
df_weighted = df[df["Data"].str.contains("weighted", case=False, na=False)].copy()
df_weighted["Exp_Type"] = df_weighted["Data"].apply(extract_experiment_type)

# For analyses by datetime_str on weighted averages, we further restrict:
weighted_vals = [
    "weighted_average_test_epsilon_scores",
    "weighted_average_test_feature_scores",
    "weighted_average_test_xper_scores"
]
df_weighted_sel = df[df["Data"].isin(weighted_vals)].copy()


In [71]:
# %% [code]
pure_counts = df[df["Pure Cluster"] == True].groupby(["cluster_method", "Exp_Type"]).size().reset_index(name="Count")
fig3 = px.bar(pure_counts, x="cluster_method", y="Count", color="Exp_Type", barmode="group",
              title="Count of Pure Clusters per Cluster Method & Experiment Type")
fig3.show()


In [89]:
# %% [code]
# Normalize metrics between 0 and 1
df_weighted_sel_norm = df_weighted_sel.copy()
for metric in metrics:
    min_val = df_weighted_sel[metric].min()
    max_val = df_weighted_sel[metric].max()
    df_weighted_sel_norm[metric] = (df_weighted_sel[metric] - min_val) / (max_val - min_val)

# Compute margin differences after normalizing
margin_records = []
for dt, group in df_weighted_sel_norm.groupby("datetime_str"):
    for metric in metrics:
        group_sorted = group.sort_values(by=metric, ascending=False)
        if len(group_sorted) >= 3:
            best = group_sorted.iloc[0][metric]
            second = group_sorted.iloc[1][metric]
            third = group_sorted.iloc[2][metric]
            margin_12 = abs(best - second)
            margin_13 = abs(best - third)
            margin_records.append({"datetime_str": dt, "Metric": metric, "Margin_1_2": margin_12, "Margin_1_3": margin_13})
margins_df = pd.DataFrame(margin_records)

# Box plot of margins
margins_melt = margins_df.melt(id_vars=["datetime_str", "Metric"], value_vars=["Margin_1_2", "Margin_1_3"],
                                var_name="Margin_Type", value_name="Margin")

fig6_fixed = px.box(margins_melt, x="Metric", y="Margin", color="Margin_Type",
                    title="Normalized Margin Between 1st, 2nd & 3rd Place (Weighted Averages)")
fig6_fixed.show()


In [79]:
# %% [code]
fig11 = make_subplots(rows=2, cols=2, subplot_titles=metrics)
for i, metric in enumerate(metrics):
    r, c = row_col[i]
    fig11.add_trace(
        go.Violin(x=df_clean["Exp_Type"], y=df_clean[metric], box_visible=True, meanline_visible=True),
        row=r, col=c
    )
    fig11.update_xaxes(title_text="Exp_Type", row=r, col=c)
    fig11.update_yaxes(title_text=metric, row=r, col=c)
fig11.update_layout(title="Violin Plots of Performance Metrics by Experiment Type (Non-Weighted)", height=800)
fig11.show()


In [80]:
# %% [code]
cluster_size_grp = df_clean.groupby(["cluster_method", "Exp_Type"])["Cluster Size"].mean().reset_index()
fig12 = px.bar(cluster_size_grp, x="cluster_method", y="Cluster Size", color="Exp_Type", barmode="group",
               title="Average Cluster Size by Cluster Method & Experiment Type")
fig12.show()


In [83]:
# %% [code]
avg_metrics = df_clean.groupby("Exp_Type")[metrics].mean().reset_index()
categories = metrics

fig15 = go.Figure()
for _, row in avg_metrics.iterrows():
    fig15.add_trace(go.Scatterpolar(
        r=[row[m] for m in categories],
        theta=categories,
        fill='toself',
        name=row["Exp_Type"]
    ))
fig15.update_layout(
    polar=dict(radialaxis=dict(visible=True)),
    title="Radar Chart: Average Performance Metrics by Experiment Type"
)
fig15.show()
